In [2]:
# use yfinance API with LLM
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown
import yfinance as yf
import json
from pprint import pprint

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

client = OpenAI()

In [3]:
# cannot use URL endpoint, using yfinance library to get stock price

def get_stock_price(symbol):
    """Get the current stock price for a given symbol."""
    try:
        # Get stock info
        stock = yf.Ticker(symbol)
        # Get current price
        current_price = stock.info['regularMarketPrice']
        return current_price
    except Exception as e:
        return f"Error getting stock price: {str(e)}"

In [4]:
# define a tool for getting stock price

tools = [
    {   'type': 'function',
        "name": "get_stock_price",
        "description": "Get the current stock price for a given symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string"
                }
            },
            "required": ["symbol"],
            'additionalProperties': False
        },
        'strict': True
    }
]

In [5]:
# first API call to extract stock symbol

input_messages = [{"role": "user", "content": "What is the stock price of BRK-B?"}]
response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools
)
print(response.output_text)

In [6]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

# call API to get stock price then join it to input_messages
result = get_stock_price(args['symbol'])
input_messages.append(tool_call)
input_messages.append({
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

In [7]:
# second API call to answer user question
response_2 = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools
)
print(response_2.output_text)

The current stock price of BRK-B is $496.98.


In [8]:
# combining the two steps above into one function

def get_stock_answer(question: str) -> str:
    input_messages = [{"role": "user", "content": question}]
    
    response = client.responses.create(
        model="gpt-4.1-nano",
        input=input_messages,
        tools=tools
    )
    
    tool_call = response.output[0]
    args = json.loads(tool_call.arguments)
    result = get_stock_price(args['symbol'])
    
    input_messages.append(tool_call)
    input_messages.append({
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    })
    
    final_response = client.responses.create(
        model="gpt-4.1-nano",
        input=input_messages,
        tools=tools
    )
    
    return final_response.output_text

In [9]:
answer = get_stock_answer("What is the PE of BRK-B?")
print(answer)

The current stock price of BRK-B is approximately $496.98. To find the PE ratio, I will need the earnings per share (EPS) for BRK-B. Typically, the PE ratio is calculated as the stock price divided by the EPS. 

Could you please provide the EPS for BRK-B or would you like me to fetch it if available?
